In [1]:
import tiktoken
import torch
import torch.nn as nn

import pandas as pd
import sys

sys.path.append('/Users/askarembulatov/Github_projects/NLP_task/NLP/NLP_Learning')

from module import *
from functions import *
from dataloader import *
from preprocessing import *


torch.manual_seed(123)
torch.set_printoptions(sci_mode=False)

/Users/askarembulatov/Github_projects/NLP_task/NLP/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("mps")

In [3]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "num_head": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "dropout": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

In [4]:
text_data = read_csv_text()

In [5]:
train_loader, val_loader = create_train_test_sample(text_data, GPT_CONFIG_124M)

In [6]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [7]:
%%time
tokenizer = tiktoken.get_encoding("gpt2")
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(123)

model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    eval_freq=5, eval_iter=5,
    start_context="who had lingered to give a lump of sugar ", num_epochs=num_epochs
)

Epoch 0 Step 0 Train loss 9.813279151916504 Val loss 9.931821823120117
Epoch 0 Step 5 Train loss 8.098200162251791 Val loss 8.310367584228516
who had lingered to give a lump of sugar , the, the,
Epoch 1 Step 10 Train loss 6.718922932942708 Val loss 7.0871076583862305
Epoch 1 Step 15 Train loss 6.066623051961263 Val loss 6.572519302368164
who had lingered to give a lump of sugar , and, and,
Epoch 2 Step 20 Train loss 5.601178010304769 Val loss 6.505828857421875
Epoch 2 Step 25 Train loss 5.344935999976264 Val loss 6.424862384796143
who had lingered to give a lump of sugar 's a him. 
Epoch 3 Step 30 Train loss 5.0728743341234 Val loss 6.447240829467773
Epoch 3 Step 35 Train loss 4.622539467281765 Val loss 6.275482177734375
who had lingered to give a lump of sugar . "I had
Epoch 4 Step 40 Train loss 4.192505439122518 Val loss 6.169416904449463
who had lingered to give a lump of sugar .    
Epoch 5 Step 45 Train loss 3.8267655902438693 Val loss 6.2545061111450195
Epoch 5 Step 50 Train loss

In [8]:
model.to("cpu")
model.eval()

tokenizer = tiktoken.get_encoding("gpt2")
token_ids = generate_text_simple(
            model=model,
            idx=text_to_token_ids("who had lingered to give a lump of sugar u", tokenizer),
            max_new_tokens=25,
            context_size=GPT_CONFIG_124M["context_length"]
            )
print("Output text:\n", token_to_text(token_ids, tokenizer))

Output text:
 who had lingered to give a lump of sugar u.




"I turned back to my work, and went on groping and muddling; then
